In [1]:
#preprocess ml
#train
#test

In [2]:
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,accuracy_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier #随机森林分类模型

In [6]:
csv_filename = "./SC_1min.csv"  # Replace with the actual filename
df = pd.read_csv(csv_filename)
df.loc[:, 'open':'hold'] = df.loc[:, 'open':'hold'].apply(pd.to_numeric, errors='coerce')
df = df.dropna()
#df.loc[:,'open':'close'] = df.loc[:,'open': 'close'].astype(float)

# df["(t+1)-(t)"] = df['close'] - df['close'].shift(-1)      # 2020-11-27修正
df["(t+1)-(t)"] = df['close'].shift(-10) - df['close']
df = df.dropna()
#print(df)
df['label'] = 0
# 构建对应表
df = df.dropna()
df['label'] = df["(t+1)-(t)"].apply(lambda x: 1 if x > 0 else 0)
df = df.dropna()
#df['datetime'] = pd.to_datetime(df['datetime']).astype(int)
df[:100]

,Unnamed: 0,datetime,open,high,low,close,volume,hold,(t+1)-(t),label
0,0.0,2022-12-13 09:18:00,510.7,510.7,510.6,510.6,5.0,11218.0,2.0,1
1,1.0,2022-12-13 09:19:00,510.6,511.2,510.5,511.2,19.0,11216.0,1.8,1
2,2.0,2022-12-13 09:20:00,511.2,511.3,511.1,511.3,3.0,11213.0,2.8,1
3,3.0,2022-12-13 09:21:00,511.3,511.8,511.1,511.1,12.0,11210.0,3.0,1
4,4.0,2022-12-13 09:22:00,511.1,511.5,511.1,511.4,4.0,11209.0,2.7,1
...,...,...,...,...,...,...,...,...,...,...
95,95.0,2022-12-13 13:31:00,517.6,520.0,517.6,520.0,12.0,10890.0,0.1,1
96,96.0,2022-12-13 13:32:00,520.0,520.0,519.4,519.6,7.0,10886.0,0.6,1
97,97.0,2022-12-13 13:33:00,519.5,519.8,517.7,519.0,28.0,10874.0,0.5,1
98,98.0,2022-12-13 13:34:00,519.0,519.8,519.0,519.5,48.0,10845.0,-0.8,0


In [4]:
class RF_predict:
    stock_code = ''
    tsData = pd.DataFrame()
    def __init__(self, stock_code):
        self.stock_code = stock_code
    def date_setting(self, start_date, end_date):
        self.tsData = df
        self.tsData = self.tsData.reset_index()
        # print(self.tsData)
    def make_predict(self, node):
        node = int(len(self.tsData) *0.8)
        #
        #print(self.tsData)
        # 构建数据集
        train_data = self.tsData[: len(self.tsData) - node]
        test_data = self.tsData[len(self.tsData) - node : ]
        train_X = train_data.loc[:, 'open': 'hold'].values
        train_y = train_data['label'].values
        
        test_X = test_data.loc[:, 'open': 'hold'].values
        test_y = test_data['label'].values

        # 进行预测
        clf = RandomForestClassifier(max_depth=10000000000, n_estimators=200)
        clf.fit(train_X, train_y)
        print(accuracy_score(train_y, clf.predict(train_X)))
        print(accuracy_score(test_y, clf.predict(test_X)))
        print(roc_auc_score(test_y,clf.predict(test_X))) # 召回率

a = RF_predict('000001')
a.date_setting(start_date='2019-05-12', end_date='2019-12-19')
a.make_predict(140)

0.9993888775718068
0.6161055365965467
0.6183498719349412


In [5]:
#test